In [2]:
import random

class Sentiment:
    POSITIVE = 'POSITIVE'
    NEGATIVE = 'NEGATIVE'

class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
    
    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVE
        else:
            return Sentiment.POSITIVE

class ReviewContainer:
    def __init__(self, reviews):
        self.reviews = reviews
        
    def get_text(self):
        return [x.text for x in self.reviews]
    
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
    
    def evenly_distribute(self):
        negative = list(filter(lambda x: x.sentiment == Sentiment.NEGATIVE, self.reviews))
        positive = list(filter(lambda x: x.sentiment == Sentiment.POSITIVE, self.reviews))
        positive_shrunk = positive[: len(negative)]
        self.reviews = positive_shrunk + negative
        random.shuffle(self.reviews)

In [3]:
import json

file_name = 'books_small_10000.json'
reviews = []

with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall']))

        
reviews[5].text

'I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia\'s trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character\'s voice on a strong subject and making it so that other peoples story may be heard through Mia\'s.'

In [4]:
len(reviews)

10000

In [5]:
from sklearn.model_selection import train_test_split

training, test = train_test_split(reviews, test_size=0.33, random_state=42)

train_container = ReviewContainer(training)
test_container = ReviewContainer(test)

train_container.evenly_distribute()
test_container.evenly_distribute()

In [6]:
len(training)

6700

In [7]:
print(training[0].sentiment)

POSITIVE


In [8]:
train_x = train_container.get_text()
train_y = train_container.get_sentiment()

In [9]:
train_x[1]

'I read it through to the end, thinking that it would get better, I was mistaken, it did not at all'

In [10]:
train_y[1]

'NEGATIVE'

In [11]:
test_x = test_container.get_text()
test_y = test_container.get_sentiment()

In [12]:
test_x[0]

"If this novel is to represent contemporary times, our modern generation, then I am quite depressed.The author appears to understand much about the drug culture and finishing antique furniture, but the supposed 'who done it' looses steam.The book launches from an interesting plot idea in the first quarter of the book, but then drowns the reader with a deluge of useless detail that fails to evolve the plot, develop the characters, or entertain. I slogged through it waiting for a curious twist and turn, given the book's hype.... and found myself bored through large sections of the storyline."

In [13]:
test_y[0]

'NEGATIVE'

In [14]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer = TfidfVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)

test_x_vectors = vectorizer.transform(test_x)

print(train_x_vectors[0].toarray())

[[0. 0. 0. ... 0. 0. 0.]]


In [15]:
train_x_vectors

<872x8900 sparse matrix of type '<class 'numpy.float64'>'
	with 53552 stored elements in Compressed Sparse Row format>

In [16]:
test_x_vectors

<416x8900 sparse matrix of type '<class 'numpy.float64'>'
	with 24256 stored elements in Compressed Sparse Row format>

In [17]:
test_x_vectors

<416x8900 sparse matrix of type '<class 'numpy.float64'>'
	with 24256 stored elements in Compressed Sparse Row format>

In [18]:
test_x_vectors[0]

<1x8900 sparse matrix of type '<class 'numpy.float64'>'
	with 68 stored elements in Compressed Sparse Row format>

In [19]:
from sklearn import svm

In [20]:
clf_svm = svm.SVC(kernel='linear')
clf_svm.fit(train_x_vectors, train_y)

SVC(kernel='linear')

In [21]:
test_x[0]

"If this novel is to represent contemporary times, our modern generation, then I am quite depressed.The author appears to understand much about the drug culture and finishing antique furniture, but the supposed 'who done it' looses steam.The book launches from an interesting plot idea in the first quarter of the book, but then drowns the reader with a deluge of useless detail that fails to evolve the plot, develop the characters, or entertain. I slogged through it waiting for a curious twist and turn, given the book's hype.... and found myself bored through large sections of the storyline."

In [22]:
test_y[0]

'NEGATIVE'

In [23]:
print(clf_svm.predict(test_x_vectors[0]))

['NEGATIVE']


In [24]:
user_input = "Overall an average book i would say. Did not enjoy much"

In [25]:
user_input_vector = vectorizer.transform([user_input])

In [26]:
print(user_input_vector)

  (0, 8813)	0.23722381445255136
  (0, 6867)	0.32723758779618994
  (0, 5601)	0.4448494139840391
  (0, 5404)	0.16369842091987755
  (0, 5220)	0.25248161051378204
  (0, 2679)	0.3333766774556179
  (0, 2221)	0.26934732589275684
  (0, 982)	0.13074200317914544
  (0, 659)	0.5483483256097684
  (0, 406)	0.21618958480328404


In [27]:
print(clf_svm.predict(user_input_vector))

['POSITIVE']


In [28]:
print(clf_svm.predict(test_x_vectors))

['NEGATIVE' 'POSITIVE' 'NEGATIVE' 'POSITIVE' 'POSITIVE' 'POSITIVE'
 'NEGATIVE' 'NEGATIVE' 'NEGATIVE' 'NEGATIVE' 'NEGATIVE' 'POSITIVE'
 'POSITIVE' 'POSITIVE' 'POSITIVE' 'POSITIVE' 'POSITIVE' 'NEGATIVE'
 'NEGATIVE' 'POSITIVE' 'NEGATIVE' 'NEGATIVE' 'NEGATIVE' 'NEGATIVE'
 'POSITIVE' 'POSITIVE' 'NEGATIVE' 'NEGATIVE' 'NEGATIVE' 'NEGATIVE'
 'POSITIVE' 'NEGATIVE' 'NEGATIVE' 'POSITIVE' 'NEGATIVE' 'NEGATIVE'
 'POSITIVE' 'POSITIVE' 'POSITIVE' 'NEGATIVE' 'POSITIVE' 'POSITIVE'
 'NEGATIVE' 'NEGATIVE' 'POSITIVE' 'POSITIVE' 'POSITIVE' 'NEGATIVE'
 'POSITIVE' 'POSITIVE' 'NEGATIVE' 'NEGATIVE' 'POSITIVE' 'NEGATIVE'
 'POSITIVE' 'POSITIVE' 'NEGATIVE' 'NEGATIVE' 'POSITIVE' 'NEGATIVE'
 'NEGATIVE' 'NEGATIVE' 'NEGATIVE' 'POSITIVE' 'NEGATIVE' 'NEGATIVE'
 'POSITIVE' 'NEGATIVE' 'NEGATIVE' 'POSITIVE' 'POSITIVE' 'NEGATIVE'
 'POSITIVE' 'NEGATIVE' 'NEGATIVE' 'NEGATIVE' 'POSITIVE' 'POSITIVE'
 'NEGATIVE' 'NEGATIVE' 'NEGATIVE' 'POSITIVE' 'POSITIVE' 'POSITIVE'
 'POSITIVE' 'NEGATIVE' 'POSITIVE' 'NEGATIVE' 'NEGATIVE' 'POSIT

In [29]:
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vectors, train_y)

DecisionTreeClassifier()

In [30]:
print(clf_dec.predict(user_input_vector[0]))

['NEGATIVE']


In [31]:
from sklearn.naive_bayes import GaussianNB

clf_gnb = GaussianNB()
clf_gnb.fit(train_x_vectors.todense(), train_y)

GaussianNB()

In [32]:
print(clf_gnb.predict(user_input_vector.todense()))

['NEGATIVE']


In [33]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression()
clf_log.fit(train_x_vectors, train_y)

LogisticRegression()

In [34]:
print(clf_log.predict(test_x_vectors[2]))

['NEGATIVE']


In [35]:
# Mean Accuracy


print(clf_svm.score(test_x_vectors, test_y))
print(clf_dec.score(test_x_vectors, test_y))
print(clf_gnb.score(test_x_vectors.todense(), test_y))
print(clf_log.score(test_x_vectors, test_y))

0.7716346153846154
0.6105769230769231
0.6394230769230769
0.7740384615384616


In [36]:
# F1 Scores

from sklearn.metrics import f1_score

print(f1_score(test_y, clf_svm.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE]))
print(f1_score(test_y, clf_dec.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE]))
print(f1_score(test_y, clf_gnb.predict(test_x_vectors.todense()), average=None, labels=[Sentiment.POSITIVE,Sentiment.NEGATIVE]))
print(f1_score(test_y, clf_log.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE]))

[0.76658477 0.77647059]
[0.61057692 0.61057692]
[0.64454976 0.63414634]
[0.77294686 0.77511962]


In [37]:
test_y.count(Sentiment.POSITIVE)

208

In [38]:
test_set = ['good read', 'bad book do not buy', 'horrible waste of time', 'Overall a mild book i would say. Did not enjoy much']
new_test = vectorizer.transform(test_set)

In [39]:
print(clf_log.predict(new_test))

['POSITIVE' 'NEGATIVE' 'NEGATIVE' 'NEGATIVE']


In [40]:
from sklearn.model_selection import GridSearchCV

parameters = {'solver': ('newton-cg', 'lbfgs','liblinear','sag','saga'), 'C': (1, 2, 3, 4, 8), 'max_iter': (400, 800,1000,1600)}
log_reg = LogisticRegression()

clf = GridSearchCV(log_reg, parameters, cv=5)

clf.fit(train_x_vectors, train_y)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': (1, 2, 3, 4, 8),
                         'max_iter': (400, 800, 1000, 1600),
                         'solver': ('newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga')})

In [41]:
print(clf.score(test_x_vectors, test_y))

0.7836538461538461


In [42]:
print(f1_score(test_y, clf.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE]))

[0.784689  0.7826087]


In [43]:
import pickle

with open('sentiment_classifier.pkl', 'wb') as f:
    pickle.dump(clf, f)

In [44]:
with open('sentiment_classifier.pkl', 'rb') as f:
    loaded_clf = pickle.load(f)

In [45]:
print(test_y[3])

print(loaded_clf.predict(test_x_vectors[3]))

POSITIVE
['POSITIVE']
